In [1]:
import pandas as pd
import numpy as np

In [2]:
smokefire = pd.read_csv('../scratch_data/MTBS_merge_smoke_inProgress.csv')

In [8]:
kaggle = pd.read_csv('../scratch_data/pollution_us_2000_2016_orig.csv')

##### New dataset found.  Filtered to pertinent info at bottom of this notebook and disseminated to group to see if we want to work with this to incorporate PM2.5.

In [3]:
#new df with PM2.5 data, though by heavy metal compounds instead of general AQI (?)
epa25 = pd.read_csv('../scratch_data/epa_hap_daily_summary.csv')

## Basic feng shui for EDA
Notes for methodology:
- AQI labels generated based on the daily maximum measurement. Air Qual will be reported as "UP TO xyz level of concern"
- Ozone measurements converted from ppm to ppb, per most common units

In [ ]:
kaggle.head()

In [223]:
kaggle.columns = kaggle.columns.str.lower().str.replace(' ','_')
kaggle.drop(columns=['unnamed:_0', 'state_code', 'county_code'], inplace=True)

In [224]:
# pd.set_option('display.max_columns', None)
# kaggle.head()

In [225]:
print(len(kaggle), len(kaggle.drop_duplicates(inplace=False))) #5k dupes, drop
kaggle.drop_duplicates(inplace=True)

1746661 1741629


Note: many Air Quality Indices consider O3 in units ppb.  This dataset uses ppm.  Converting to ppb (ppm * 1000) to make it more readable for us, and on the same scale as other pollutants.

In [226]:
kaggle['o3_mean'] = kaggle['o3_mean'] * 1000
kaggle['o3_1st_max_value'] = kaggle['o3_1st_max_value'] * 1000

In [227]:
#consolidate unit & measurement cols
kaggle.rename(columns = {'no2_mean': 'no2_mean_ppb', 'no2_1st_max_value': 'no2_max_ppb', 'no2_1st_max_hour':'no2_max_hr_ppb',
                        'o3_mean':'o3_mean_ppb', 'o3_1st_max_value':'o3_max_ppb', 'o3_1st_max_hour':'o3_max_hr_ppb',
                        'so2_mean':'so2_mean_ppb','so2_1st_max_value':'so2_max_ppb', 'so2_1st_max_hour':'so2_max_hr_ppb',
                        'co_mean':'co_mean_ppm', 'co_1st_max_value':'co_max_ppm', 'co_1st_max_hour':'co_max_hr_ppm'},
             inplace=True)
kaggle.drop(columns = ['so2_units','co_units','o3_units','no2_units'], inplace=True)

In [228]:
keep_states = ['California', 'Washington', 'Oregon', 'Arizona', 'Nevada', 'Idaho', 'Utah']
kaggle = kaggle.loc[kaggle['state'].isin(keep_states)]

In [229]:
#mapping each pollutant value to it's AQI index 'class'

def co_aqi_label(x): #for co, map to co_max_ppm column (NOT HOUR)
    if x < 4.5:
        return 'good'
    if x < 9.5:
        return 'moderate'
    if x < 12.5:
        return 'unhealthy for at-risk'
    if x < 15.5:
        return 'unhealthy'
    if x < 30.5:
        return 'very unhealthy'
    if np.isnan(x) == True:
        return np.nan
    else:
        return 'nope'
    

def no2_aqi_label(x): #map to no2_max, o3_max (NOT HOUR)
    if x < 54:
        return 'good'
    if x < 101:
        return 'moderate'
    if x < 361:
        return 'unhealthy for at-risk'
    if x < 650:
        return 'unhealthy'
    if x < 1250:
        return 'very unhealthy'
    if x < 1650:
        return 'hazardous'
    if np.isnan(x) == True:
        return np.nan
    else:
        return 'nope'
    
def so2_aqi_label(x): #map to so2_max (NOT HOUR)
    if x < 36:
        return 'good'
    if x < 76:
        return 'moderate'
    if x < 186:
        return 'unhealthy for at-risk'
    if x < 305:
        return 'unhealthy'
    if x < 605:
        return 'very unhealthy'
    if x < 805:
        return 'hazardous'
    if np.isnan(x) == True:
        return np.nan
    else:
        return 'nope'

#o3_aqi_labels units in ppb **
def o3_aqi_label(x): #mishmash of 1hr and 8hr classification regimens
    if x < 55:
        return 'good'
    if x < 71:
        return 'moderate'
    if x < 86:
        return 'unhealthy for at-risk'
    if x < 106:
        return 'unhealthy'
    if x < 500:
        return 'hazardous'
    if x > 500:
        return 'nope'
    if np.isna(x) == True:
        return np.nan

In [230]:
kaggle['co_aqi_label'] = kaggle['co_max_ppm'].map(lambda x: co_aqi_label(x))

so2_col = kaggle['so2_max_ppb'].map(lambda x: so2_aqi_label(x))
kaggle.insert(17, 'so2_aqi_label', so2_col) 

o3_col = kaggle['o3_max_ppb'].map(lambda x: o3_aqi_label(x))
kaggle.insert(13, 'o3_aqi_label', o3_col) 

no2_col = kaggle['no2_max_ppb'].map(lambda x: no2_aqi_label(x))
kaggle.insert(9, 'no2_aqi_label', no2_col) 

#testcopy.insert(10, 'no2_aqi2', new_col)

In [231]:
kaggle.reset_index(drop=True, inplace=True)

In [232]:
#kaggle.to_csv('../scratch_data/pollution_us_2000_2016.csv', index=False)

## On to merging
* smokefire county_name all end in 'County'; kaggle 'county' column does not.  Add 'County' to end of kaggle counties.  And change kaggle col name to 'county_name'.
* kaggle date = object, change to int

In [4]:
kaggle = pd.read_csv('../scratch_data/pollution_us_2000_2016.csv')

In [5]:
smokefire.head(1)

,state,county_name,date,smoke_score,lat_smo,lon_smo,event_id,incid_name,incid_type,asmnt_type,...,nodata_t,incgreen_t,low_t,mod_t,high_t,geometry,lon_wf,lat_wf,reverse_geocode_fire_ig,state_full
0,AZ,Gila County,20100604,1.0,33.799665,-110.811659,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
kaggle.head()

,site_num,address,state,county,city,date_local,no2_mean_ppb,no2_max_ppb,no2_max_hr_ppb,no2_aqi_label,...,so2_mean_ppb,so2_max_ppb,so2_max_hr_ppb,so2_aqi_label,so2_aqi,co_mean_ppm,co_max_ppm,co_max_hr_ppm,co_aqi,co_aqi_label
0,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,good,...,3.000000,9.0,21,good,13.0,1.145833,4.2,21,NaN,good
1,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,good,...,3.000000,9.0,21,good,13.0,0.878947,2.2,23,25.0,good
2,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,good,...,2.975000,6.6,23,good,NaN,1.145833,4.2,21,NaN,good
3,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,19.041667,49.0,19,good,...,2.975000,6.6,23,good,NaN,0.878947,2.2,23,25.0,good
4,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,22.958333,36.0,19,good,...,1.958333,3.0,22,good,4.0,0.850000,1.6,23,NaN,good


In [7]:
k_county_col = kaggle['county'] + ' County'
kaggle.insert(3,'county_name', k_county_col) 

In [8]:
k_date = kaggle['date_local'].str.replace('-','').astype('int')
kaggle.insert(5, 'date', k_date)

In [9]:
state_dict = {'Idaho':'ID', 'California': 'CA', 'Arizona':'AZ', 'Oregon':'OR', 
              'Washington':'WA', 'Nevada':'NV', 'Utah':'UT'}

kaggle['state'] = kaggle['state'].map(state_dict)

In [12]:
kaggle.head(1)

,site_num,address,state,county_name,county,date,city,date_local,no2_mean_ppb,no2_max_ppb,...,so2_mean_ppb,so2_max_ppb,so2_max_hr_ppb,so2_aqi_label,so2_aqi,co_mean_ppm,co_max_ppm,co_max_hr_ppm,co_aqi,co_aqi_label
0,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,AZ,Maricopa County,Maricopa,20000101,Phoenix,2000-01-01,19.041667,49.0,...,3.0,9.0,21,good,13.0,1.145833,4.2,21,NaN,good


In [13]:
kaggle.drop(columns=['county', 'date_local'], inplace=True)

In [14]:
kagformerge = kaggle.loc[kaggle['date']>=20100604].copy()

In [15]:
kagformerge.drop(columns=['site_num','no2_mean_ppb', 'no2_max_hr_ppb', 'o3_mean_ppb',
                         'o3_max_hr_ppb', 'so2_mean_ppb', 'so2_max_hr_ppb', 'co_mean_ppm', 
                         'co_max_hr_ppm'], inplace=True)

kagformerge.rename(columns={'address':'aqi_address'},inplace=True)

In [16]:
kagformerge.shape, smokefire.shape

((246308, 17), (83899, 24))

In [17]:
print(min(kagformerge.date), max(kagformerge.date))
print(min(smokefire.date), max(smokefire.date))

20100604 20160430
20100604 20161221


In [18]:
smokeformerge = smokefire.loc[smokefire['date']<=20160430].copy()

In [19]:
kagformerge.drop_duplicates(inplace=True)

In [20]:
print(min(kagformerge.date), max(kagformerge.date))
print(min(smokeformerge.date), max(smokeformerge.date))

20100604 20160430
20100604 20160428


In [28]:
# pd.set_option('display.max_columns',None)
# smokeformerge.head()

In [24]:
smokeformerge.drop(columns=['state_full', 'asmnt_type', 'lat_wf', 'lon_wf', 'nodata_t', 'incgreen_t',
                            'low_t', 'mod_t', 'high_t'], inplace=True)
smokeformerge.drop_duplicates(inplace=True)

In [26]:
smokeformerge.rename(columns={'geometry':'geometry_wf'}, inplace=True)

In [29]:
len(smokefire), len(smokeformerge) #from dropping dupes after dropping the above cols ^^ hmmm

(83899, 73187)

In [30]:
all_smo_fire_poll = pd.merge(smokeformerge, kagformerge, on=['state', 'county_name', 'date'], how='outer')
## inner join loses 70% of fires, I checked
#inner_smo_fire_poll = pd.merge(smokeformerge, kagformerge, on=['state', 'county_name', 'date'], how='inner')
#preserve all fire data, not pollution:
mostlysmo = pd.merge(smokeformerge, kagformerge, on=['state', 'county_name', 'date'], how='left')

In [32]:
all_smo_fire_poll.shape, inner_smo_fire_poll.shape, mostlysmo.shape

((321580, 29), (26400, 29), (95303, 29))

In [35]:
#inner_smo_fire_poll = inner_smo_fire_poll[['state', 'city', 'county_name', 'date', 'incid_type', 'burnbndac',
                                          'smoke_score', 'no2_max_ppb', 'no2_aqi_label', 'o3_max_ppb', 'o3_aqi_label',
                                          'so2_max_ppb', 'so2_aqi_label', 'co_max_ppm', 'co_aqi_label', 'incid_name',
                                          'event_id', 'burnbndlat', 'burnbndlon', 'lat_smo', 'lon_smo',
                                          'aqi_address', 'geometry_wf','reverse_geocode_fire_ig']]

all_smo_fire_poll = all_smo_fire_poll[['state', 'city', 'county_name', 'date', 'incid_type', 'burnbndac',
                                          'smoke_score', 'no2_max_ppb', 'no2_aqi_label', 'o3_max_ppb', 'o3_aqi_label',
                                          'so2_max_ppb', 'so2_aqi_label', 'co_max_ppm', 'co_aqi_label', 'incid_name',
                                          'event_id', 'burnbndlat', 'burnbndlon', 'lat_smo', 'lon_smo',
                                          'aqi_address', 'geometry_wf','reverse_geocode_fire_ig']]

mostlysmo = mostlysmo[['state', 'city', 'county_name', 'date', 'incid_type', 'burnbndac',
                                          'smoke_score', 'no2_max_ppb', 'no2_aqi_label', 'o3_max_ppb', 'o3_aqi_label',
                                          'so2_max_ppb', 'so2_aqi_label', 'co_max_ppm', 'co_aqi_label', 'incid_name',
                                          'event_id', 'burnbndlat', 'burnbndlon', 'lat_smo', 'lon_smo',
                                          'aqi_address', 'geometry_wf','reverse_geocode_fire_ig']]

In [36]:
all_smo_fire_poll.drop_duplicates(inplace=True)
mostlysmo.drop_duplicates(inplace=True)
#inner_smo_fire_poll.drop_duplicates(inplace=True)

<ipython-input-36-3304f0d1bc64>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inner_smo_fire_poll.drop_duplicates(inplace=True)


In [37]:
all_smo_fire_poll.reset_index(drop=True, inplace=True)
mostlysmo.reset_index(drop=True, inplace=True)
#inner_smo_fire_poll.reset_index(drop=True, inplace=True)

In [42]:
all_smo_fire_poll.to_csv('../scratch_data/merges/all_smoke_fire_pollution.csv', index=False)
mostlysmo.to_csv('../scratch_data/merges/matched_smoke_fire_pollution.csv', index=False)
#inner_smo_fire_poll.to_csv('../scratch_data/merges/smoke-fire_inner_poll.csv', index=False)

## Filter epa data to States, dates, and pollutants of interest; see if the group wants to work with this

In [5]:
keep_states = ['California', 'Washington', 'Oregon', 'Arizona', 'Nevada', 'Idaho', 'Utah']

epa25 = epa25.loc[epa25['state_name'].isin(keep_states)]

In [6]:
epa25 = epa25.loc[epa25['parameter_name'].str.contains('2.5')]

In [7]:
epa25 = epa25.loc[epa25['date_local']>='2010-01-01']

In [69]:
# epa25.to_csv('../scratch_data/pm2.5_epa_2010_2017.csv', index=False)